In [2]:
import pandas as pd

In [4]:
df = pd.read_csv("spotify_millsongdata.csv")

In [6]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [8]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [10]:
df.shape

(57650, 4)

In [12]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [14]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [16]:
df.head(10)

,artist,song,text
0,Loretta Lynn,I've Got A Picture Of Us On My Mind,In a hot dusty bar room in town \r\nThat old ...
1,Michael Jackson,In The Closet,"She's just a lover, who makes me high \r\nIs ..."
2,Kylie Minogue,No World Without You,The sweet perfume of flowers in bloom \r\nRem...
3,Queen,Bicycle Race,Bicycle bicycle bicycle \r\nI want to ride my...
4,Kim Wilde,Try Again,I can't see your face \r\nHear the excuses fo...
5,Johnny Cash,Away In A Manger,"Away in a manger, no crib for His bed \r\nThe..."
6,Janis Joplin,Coo-Coo ( In Album Cheaper Thrills ),"Oh, the cuckoo, she's a pretty bird, and she w..."
7,XTC,Pink Thing,"Anytime you rise, I'm here, \r\nAnd I'm crazy..."
8,Drake,Back To Back,"[Intro] \r\nOh man, oh man, oh man, not again..."
9,Yeah Yeah Yeahs,Honeybear,Turn yourself around! \r\nYou weren't invited...


In [18]:
df['text'][0]

"In a hot dusty bar room in town  \r\nThat old jukebox plays the honky tonk sound  \r\nYou might think that I'm crying, Lord I am  \r\nBut I'm trying to get over what's over and through  \r\nI've got a picture of us on my mind  \r\nSweet, sweet scenes of once upon a time  \r\nI can't picture myself with anyone else  \r\nI've got a picture of us on my mind  \r\nFrom the pockets of his ragged old jeans  \r\nHe paid the preacher after we said I do  \r\nI think of good times we've known  \r\nThat were as good as he's gone  \r\nThis glass of red wine feels good going down  \r\nI've got a picture of us on my mind  \r\nSweet, sweet scenes of once upon a time  \r\nI can't picture myself with anyone else  \r\nI've got a picture of us on my mind  \r\nI've got a picture of us on my mind  \r\nSweet, sweet scenes of once upon a time  \r\nI can't picture myself with anyone else  \r\nI've got a picture of us on my mind  \r\n\r\n"

In [20]:
df.shape

(5000, 3)

In [22]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [24]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [26]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [30]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [32]:
similarity[0]

array([1.        , 0.00435359, 0.10673927, ..., 0.01510052, 0.00146885,
       0.        ])

In [44]:
df[df['song'] == 'In The Closet']

,artist,song,text
1,Michael Jackson,In The Closet,"she 's just a lover , who make me high is it w..."


In [40]:
def recommendation(song_df):
    if song_df not in df['song'].values:
        return f"The song '{song_df}' is not in the dataset."
    
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs


In [46]:
recommendation('In The Closet')

['I Just Want To Make Love To You',
 'Home Tonight',
 'Over And Over',
 'I Want To Die A Beautiful Death',
 "I Don't Wanna Talk About It Now",
 'Love You Madly',
 'I Want You So Bad',
 "Don't You Want To Be There",
 'Fake Diamonds',
 'Original Of The Species',
 'As Long As You Hold Me',
 "Where There's No Tomorrow",
 'All I Want',
 "I Can't Be",
 'House On Fire',
 'I Want You Now',
 "I Don't Want To Get Hurt",
 'All I Wanna Do Is Make Love To You',
 'Low Down',
 'Hero']

In [48]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))